In [14]:
import requests
import zipfile
from arcpy.sa import *
import os

In [15]:
#Create path for data
path = 'C:/Users/tjjoh/Documents/GIS5571/PRISMdata'
os.mkdir(path)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:/Users/tjjoh/Documents/GIS5571/PRISMdata'

In [16]:
#Create base directories for each month of the year
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
url_base = 'https://services.nacse.org/prism/data/public/normals/4km/ppt/{}'
zip_base = 'C:/Users/tjjoh/Documents/GIS5571/PRISMdata/{}.zip'
file_base = 'C:/Users/tjjoh/Documents/GIS5571/PRISMdata/{}'

In [17]:
#Make an API call to get data for each month and unzip the .bil files
for month in months:
    url = url_base.format(month)
    response = requests.get(url)
    
    with open(zip_base.format(month), 'wb') as file:
        file.write(response.content)
        
    with zipfile.ZipFile(zip_base.format(month), 'r') as zip_ref:
        zip_ref.extractall(file_base.format(month))

In [18]:
#Convert .bil files to TIFF
arcpy.conversion.RasterToOtherFormat(
    Input_Rasters=r"C:\Users\tjjoh\Documents\GIS5571\PRISMdata\01\PRISM_tmax_30yr_normal_4kmM5_01_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\02\PRISM_tmax_30yr_normal_4kmM5_02_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\03\PRISM_tmax_30yr_normal_4kmM5_03_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\04\PRISM_tmax_30yr_normal_4kmM5_04_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\05\PRISM_tmax_30yr_normal_4kmM5_05_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\06\PRISM_tmax_30yr_normal_4kmM5_06_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\07\PRISM_tmax_30yr_normal_4kmM5_07_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\08\PRISM_tmax_30yr_normal_4kmM5_08_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\09\PRISM_tmax_30yr_normal_4kmM5_09_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\10\PRISM_tmax_30yr_normal_4kmM5_10_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\11\PRISM_tmax_30yr_normal_4kmM5_11_bil.bil;C:\Users\tjjoh\Documents\GIS5571\PRISMdata\12\PRISM_tmax_30yr_normal_4kmM5_12_bil.bil",
    Output_Workspace="C:/Users/tjjoh/Documents/GIS5571/PRISMdata/tiffs",
    Raster_Format="TIFF"
)

<Result ''>

In [19]:
#Create an empty mosaic dataset
arcpy.management.CreateMosaicDataset(
    in_workspace=r"C:\Users\tjjoh\Documents\GIS5571\Lab2_GDB.gdb",
    in_mosaicdataset_name="Mosaic",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)

<Result 'C:\\Users\\tjjoh\\Documents\\GIS5571\\Lab2_GDB.gdb\\Mosaic'>

In [20]:
#Add the TIFFs to the mosaic dataset
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset="Mosaic",
    raster_type="Raster Dataset",
    input_path=r"C:\Users\tjjoh\Documents\GIS5571\PRISMdata\tiffs",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="NO_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="NO_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\tjjoh\AppData\Local\ESRI\rasterproxies\Mosaic"
)

<Result 'Mosaic'>

In [21]:
#Add timestamp to mosaic dataset
arcpy.management.CalculateField(
    in_table=r"Mosaic\Footprint",
    field="Timestamp",
    expression="DateAdd(Date(1991,0,1), $feature.OBJECTID-1,'year')",
    expression_type="ARCADE",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Mosaic\\Footprint'>

In [22]:
#Add variable to mosaic dataset
arcpy.management.CalculateField(
    in_table=r"Mosaic\Footprint",
    field="Variable",
    expression='"Precipitation"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Mosaic\\Footprint'>

In [23]:
#Add multidimensional metadata to mosaic dataset
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset="Mosaic",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'Mosaic'>

In [24]:
#Convert mosaic dataset to multidimensional raster layer
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="Mosaic",
    out_multidimensional_raster_layer="Mosaic_MultidimLayer",
    variables="Precipitation",
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-125.020849256 24.0625101097931 -66.479182588865 49.9375101100001 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
)

<Result 'Mosaic_MultidimLayer'>

In [25]:
#Create space time cube
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="Mosaic_MultidimLayer",
    output_cube=r"C:\Users\tjjoh\Documents\GIS5571\PRISMdata\PRISMstc.nc",
    fill_empty_bins="ZEROS"
)

<Result 'C:\\Users\\tjjoh\\Documents\\GIS5571\\PRISMdata\\PRISMstc.nc'>